In [132]:
!pip3 install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [1]:
import joblib
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [36]:
def processData(coin):
  #Reading data
  BASE_URL = '../data/'
  data = pd.read_csv(BASE_URL + coin + '.csv')

  #Creating date
  data['Date'] = pd.to_datetime(data['timestamp'], unit = 's').dt.date

  #Intializing variable
  dates = data['Date'].unique()
  dates = list(dates)
  for i in range(len(dates)):
    dates[i] = str(dates[i])

  #Extracting ['Date', 'Close'] and setting 'date' as index
  data = data[['Close', 'Date']]
  data['Date'] = data['Date'].astype(str)

  final_data = []

  #Creating day-wise dataframe for the given data
  for i in dates:
    max = data[data['Date'] == i][-1:]
    final_data.append(max)

  #Reshape data to 2D
  final_data = np.array(final_data).reshape(len(final_data), 2)

  #Converting to DataFrame
  cols = data.columns
  final_data = pd.DataFrame(final_data, columns = cols)

  return final_data.set_index('Date', drop = True)

def preprocessData(dataCleaned):
  data_matrix = []
  col = 10

  # Normalizing data
  scaler = MinMaxScaler()
  dataCleaned['Close'] = dataCleaned['Close'].astype(float)
  dataCleaned['Close'] = scaler.fit_transform(np.array(dataCleaned['Close']).reshape(-1,1))

  for i in range(len(dataCleaned) - col+1):
    data_matrix.append(dataCleaned[i:i+col].values)

  row = int(len(data_matrix) * 0.7)
  dataCleaned = np.asarray(data_matrix)

  # # Splitting data
  X_train, y_train = dataCleaned[:row, :-1], dataCleaned[:row, -1]
  X_test, y_test = dataCleaned[row:, :-1], dataCleaned[row:, -1]

  # Reshaping feature variables
  # 1. X_train
  shape = list(X_train.shape)
  totShape = np.prod(shape)
  main = int(totShape/9)

  X_train = np.array(X_train).reshape(main, 1, 9)

  # 2. X_test
  shape = list(X_test.shape)
  totShape = np.prod(shape)
  main = int(totShape/9)

  X_test = np.array(X_test).reshape(main, 1, 9)

  return scaler, X_train, y_train, X_test, y_test

def model_train(X_train, y_train, coin):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.LSTM(units = 30, input_shape = (None, 9), return_sequences = True))
  model.add(tf.keras.layers.Dense(units = 32, activation = 'linear'))
  model.add(tf.keras.layers.LSTM(units = 30, return_sequences = False))
  model.add(tf.keras.layers.Dense(units = 1, activation = 'linear'))

  model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mse'])

  model.fit(X_train, y_train, batch_size = 5, epochs = 25, validation_split = .02)
  
  path = f'../models/{coin}.h5'
  model.save(f'../models/{coin}.h5')
  
  return f'Model saved successfully at {path}!'

from keras.models import load_model

def predict_NextDay(scaler, coin):
  MODEL_PATH = f'../models/{coin}.h5' 
  model = load_model(MODEL_PATH)
  coins_dict = {
    'dogecoin' : 'DOGE-USD',
    'bitcoin' : 'BTC-USD',
    'ethereum' : 'ETH-USD',
    'iota' : 'MIOTA-USD',
    'tron' : 'TRX-USD'
  }

  ticker = coins_dict[coin]

  today = datetime.date.today()
  d1 = today.strftime("%Y-%m-%d")
  d2 = datetime.date.today() - datetime.timedelta(days = 9)
  d2 = d2.strftime("%Y-%m-%d")

  new_sample = yf.download(ticker, start = d2, end = d1, progress = False)

  last9 = new_sample[['Close']]
  vals = last9.values

  temp = []

  for i in range(len(vals)):
    temp.append(vals[0][0])

  sample = scaler.fit_transform(np.array(temp).reshape(-1, 1))
  sample = np.asarray(sample)[:-1]

  sample = np.reshape(sample, (1, 1, 9))

  prediction = model.predict(sample, batch_size = 2)

  prediction = scaler.inverse_transform(prediction)
  return prediction

In [37]:
coins_dict = {
    'dogecoin' : 'DOGE-USD',
    'bitcoin' : 'BTC-USD',
    'ethereum' : 'ETH-USD',
    'iota' : 'MIOTA-USD',
    'tron' : 'TRX-USD'
}

In [38]:
coins_dict['dogecoin']

'DOGE-USD'

In [28]:
data = processData('dogecoin')

In [29]:
scaler, X_train, y_train, X_test, y_test = preprocessData(data)

In [30]:
model_train(X_train, y_train, 'dogecoin')

Epoch 1/25
122/122 [==============================] - 3s 6ms/step - loss: 8.6577e-07 - mse: 8.6577e-07 - val_loss: 6.9689e-06 - val_mse: 6.9689e-06
Epoch 2/25
122/122 [==============================] - 0s 2ms/step - loss: 5.2034e-07 - mse: 5.2034e-07 - val_loss: 2.8727e-06 - val_mse: 2.8727e-06
Epoch 3/25
122/122 [==============================] - 0s 2ms/step - loss: 6.9385e-07 - mse: 6.9385e-07 - val_loss: 1.2450e-05 - val_mse: 1.2450e-05
Epoch 4/25
122/122 [==============================] - 0s 2ms/step - loss: 8.0624e-07 - mse: 8.0624e-07 - val_loss: 3.8912e-06 - val_mse: 3.8912e-06
Epoch 5/25
122/122 [==============================] - 0s 2ms/step - loss: 6.3938e-07 - mse: 6.3938e-07 - val_loss: 3.0363e-06 - val_mse: 3.0363e-06
Epoch 6/25
122/122 [==============================] - 0s 2ms/step - loss: 6.8020e-07 - mse: 6.8020e-07 - val_loss: 4.0604e-06 - val_mse: 4.0604e-06
Epoch 7/25
122/122 [==============================] - 0s 2ms/step - loss: 5.8600e-07 - mse: 5.8600e-07 - val_los

'Model saved successfully at ../models/dogecoin.h5!'

In [39]:
predict_NextDay(scaler, 'dogecoin')

array([[0.08195435]], dtype=float32)

In [40]:
#Bitcoin
coin = 'bitcoin'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
186/186 [==============================] - 3s 5ms/step - loss: 6.1786e-04 - mse: 6.1786e-04 - val_loss: 1.6950e-04 - val_mse: 1.6950e-04
Epoch 2/25
186/186 [==============================] - 0s 2ms/step - loss: 9.6080e-05 - mse: 9.6080e-05 - val_loss: 1.7928e-04 - val_mse: 1.7928e-04
Epoch 3/25
186/186 [==============================] - 0s 2ms/step - loss: 7.2270e-05 - mse: 7.2270e-05 - val_loss: 1.4900e-04 - val_mse: 1.4900e-04
Epoch 4/25
186/186 [==============================] - 0s 2ms/step - loss: 6.4957e-05 - mse: 6.4957e-05 - val_loss: 5.6752e-05 - val_mse: 5.6752e-05
Epoch 5/25
186/186 [==============================] - 0s 2ms/step - loss: 5.4341e-05 - mse: 5.4341e-05 - val_loss: 1.4751e-04 - val_mse: 1.4751e-04
Epoch 6/25
186/186 [==============================] - 0s 2ms/step - loss: 5.1496e-05 - mse: 5.1496e-05 - val_loss: 4.0726e-05 - val_mse: 4.0726e-05
Epoch 7/25
186/186 [==============================] - 0s 2ms/step - loss: 5.3424e-05 - mse: 5.3424e-05 - val_los

array([[24319.334]], dtype=float32)

In [41]:
#Ethereum
coin = 'ethereum'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
186/186 [==============================] - 3s 5ms/step - loss: 8.9399e-04 - mse: 8.9399e-04 - val_loss: 1.6973e-04 - val_mse: 1.6973e-04
Epoch 2/25
186/186 [==============================] - 0s 2ms/step - loss: 1.2470e-04 - mse: 1.2470e-04 - val_loss: 9.6381e-05 - val_mse: 9.6381e-05
Epoch 3/25
186/186 [==============================] - 0s 2ms/step - loss: 1.1299e-04 - mse: 1.1299e-04 - val_loss: 4.9017e-05 - val_mse: 4.9017e-05
Epoch 4/25
186/186 [==============================] - 0s 2ms/step - loss: 8.6345e-05 - mse: 8.6345e-05 - val_loss: 1.4774e-04 - val_mse: 1.4774e-04
Epoch 5/25
186/186 [==============================] - 0s 2ms/step - loss: 8.4322e-05 - mse: 8.4322e-05 - val_loss: 1.1027e-04 - val_mse: 1.1027e-04
Epoch 6/25
186/186 [==============================] - 0s 2ms/step - loss: 8.8567e-05 - mse: 8.8567e-05 - val_loss: 8.1287e-05 - val_mse: 8.1287e-05
Epoch 7/25
186/186 [==============================] - 0s 2ms/step - loss: 7.3394e-05 - mse: 7.3394e-05 - val_los

array([[1936.8021]], dtype=float32)

In [42]:
#IOTA
coin = 'iota'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
168/168 [==============================] - 3s 6ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 3.8486e-04 - val_mse: 3.8486e-04
Epoch 2/25
168/168 [==============================] - 0s 2ms/step - loss: 7.0046e-04 - mse: 7.0046e-04 - val_loss: 3.7623e-04 - val_mse: 3.7623e-04
Epoch 3/25
168/168 [==============================] - 0s 2ms/step - loss: 6.2609e-04 - mse: 6.2609e-04 - val_loss: 1.2399e-04 - val_mse: 1.2399e-04
Epoch 4/25
168/168 [==============================] - 0s 2ms/step - loss: 5.4909e-04 - mse: 5.4909e-04 - val_loss: 1.1536e-04 - val_mse: 1.1536e-04
Epoch 5/25
168/168 [==============================] - 0s 2ms/step - loss: 5.1902e-04 - mse: 5.1902e-04 - val_loss: 8.2237e-05 - val_mse: 8.2237e-05
Epoch 6/25
168/168 [==============================] - 0s 2ms/step - loss: 4.8879e-04 - mse: 4.8879e-04 - val_loss: 8.6221e-05 - val_mse: 8.6221e-05
Epoch 7/25
168/168 [==============================] - 0s 2ms/step - loss: 4.8091e-04 - mse: 4.8091e-04 - val_loss: 1.178

array([[0.35421285]], dtype=float32)

In [43]:
#TRON
coin = 'tron'
data = processData(coin)
scaler, X_train, y_train, X_test, y_test = preprocessData(data)
model_train(X_train, y_train, coin)
predict_NextDay(scaler, coin)

Epoch 1/25
181/181 [==============================] - 3s 5ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 5.8136e-04 - val_mse: 5.8136e-04
Epoch 2/25
181/181 [==============================] - 0s 2ms/step - loss: 5.5091e-04 - mse: 5.5091e-04 - val_loss: 6.4725e-04 - val_mse: 6.4725e-04
Epoch 3/25
181/181 [==============================] - 0s 2ms/step - loss: 4.3056e-04 - mse: 4.3056e-04 - val_loss: 1.9550e-04 - val_mse: 1.9550e-04
Epoch 4/25
181/181 [==============================] - 0s 2ms/step - loss: 4.1160e-04 - mse: 4.1160e-04 - val_loss: 1.7619e-04 - val_mse: 1.7619e-04
Epoch 5/25
181/181 [==============================] - 0s 2ms/step - loss: 2.7299e-04 - mse: 2.7299e-04 - val_loss: 8.4099e-04 - val_mse: 8.4099e-04
Epoch 6/25
181/181 [==============================] - 0s 2ms/step - loss: 2.4025e-04 - mse: 2.4025e-04 - val_loss: 2.0195e-04 - val_mse: 2.0195e-04
Epoch 7/25
181/181 [==============================] - 0s 2ms/step - loss: 2.5381e-04 - mse: 2.5381e-04 - val_loss: 2.055

array([[0.07986443]], dtype=float32)